In [1]:
import pysam

In [3]:
def get_transcript_count(sam_file, transcript_name):

    """
    Return the count of the transcript i.e how many of each of \
    the three transcripts are in there.
    Note - Does not look at the quality of the alignment.
    """

    sam_file_location = sam_file

    samfile = pysam.AlignmentFile(sam_file_location, "rb")

    count = 0

    for read in samfile:

        if read is not None and read.reference_name == transcript_name:

            count = count + 1

    return count

In [7]:
get_transcript_count('SRR1918637.sam',1)

AttributeError: 'module' object has no attribute 'AlignmentFile'

In [8]:
dir(pysam)

['AlignedRead',
 'Fastafile',
 'IndexedReads',
 'IteratorColumn',
 'IteratorRow',
 'Pileup',
 'PileupColumn',
 'PileupProxy',
 'PileupRead',
 'SAMTOOLS_DISPATCH',
 'Samfile',
 'SamtoolsDispatcher',
 'SamtoolsError',
 'Tabixfile',
 'VCF',
 'VCFRecord',
 '__all__',
 '__builtins__',
 '__doc__',
 '__file__',
 '__name__',
 '__package__',
 '__path__',
 '__samtools_version__',
 '__version__',
 'asBed',
 'asGTF',
 'asTuple',
 'asVCF',
 'bam2fq',
 'calmd',
 'cat',
 'cmd',
 'csamtools',
 'ctabix',
 'cvcf',
 'depth',
 'faidx',
 'fixmate',
 'flagstat',
 'idxstats',
 'index',
 'key',
 'merge',
 'mpileup',
 'options',
 'os',
 'parser',
 'phase',
 'reheader',
 'rmdup',
 'samimport',
 'sort',
 'sys',
 'tabix_compress',
 'tabix_index',
 'targetcut',
 'tview',
 'version',
 'view']